In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import glob, numpy as np, pandas as pd, tensorflow as tf, matplotlib.pyplot as plt, skimage.io, os, gc, random
from tensorflow.keras import Input, Model, models, layers, optimizers, losses, callbacks, utils
from pathlib import Path
from jarvis.train import custom
from jarvis.utils.display import imshow
from skimage.transform import rescale, resize
import cv2
from jarvis.utils import arrays as jars
import SimpleITK as sitk
from scipy.interpolate import splprep, splev

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
import jerry_losses, jerry_metrics
from jerry_utils import restart_kernel, load_dataset, load_dataset_v1

[ 2025-02-13 00:24:50 ] CUDA_VISIBLE_DEVICES automatically set to: 0           


In [2]:
# VasM functions

def find_contours(segmentation):
    contours, _ = cv2.findContours(segmentation.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours[0]

def line_intersection(p1, p2, q1, q2):
    s = np.vstack([p1, p2, q1, q2])
    h = np.hstack((s, np.ones((4, 1))))
    l1 = np.cross(h[0], h[1])
    l2 = np.cross(h[2], h[3])
    x, y, z = np.cross(l1, l2)
    if z == 0:
        return (float('inf'), float('inf'))
    return (x / z, y / z)

def find_constrained_endpoints(com, axis, contour_pts, buffer=1e-6):
    axis_unit = axis / np.linalg.norm(axis)
    intersections = []

    for i in range(len(contour_pts)):
        p1 = contour_pts[i][0]
        p2 = contour_pts[(i + 1) % len(contour_pts)][0]
        intersection = line_intersection(com - 10000 * axis_unit, com + 10000 * axis_unit, p1, p2)

        min_bound = np.minimum(p1, p2) - buffer
        max_bound = np.maximum(p1, p2) + buffer

        if np.all(np.isfinite(intersection)) and np.all(min_bound <= intersection) and np.all(intersection <= max_bound):
            intersections.append(intersection)

    projections = [np.dot(intersection - com, axis_unit) for intersection in intersections]
    if not projections:
        return None, None

    min_proj = min(projections)
    max_proj = max(projections)

    min_pt = com + min_proj * axis_unit
    max_pt = com + max_proj * axis_unit

    return min_pt, max_pt

def get_centroid(arr):
    img = sitk.GetImageFromArray(arr.astype(int))
    filter_label = sitk.LabelShapeStatisticsImageFilter()
    filter_label.Execute(img)
    com_y, com_x = filter_label.GetCentroid(1)
    return np.array([com_y, com_x])

def sclerotic_index(vessel_segmentation, lumen_segmentation):
    com = get_centroid(lumen_segmentation)
    lumen_contour = find_contours(lumen_segmentation)
    vessel_contour = find_contours(vessel_segmentation)

    sclerotic_indices = []

    for angle in range(180):
        radians = np.deg2rad(angle)
        axis = np.array([np.cos(radians), np.sin(radians)])

        min_pt_lumen, max_pt_lumen = find_constrained_endpoints(com, axis, lumen_contour)
        if min_pt_lumen is None or max_pt_lumen is None:
            continue
        internal_diameter = np.linalg.norm(max_pt_lumen - min_pt_lumen)

        min_pt_vessel, max_pt_vessel = find_constrained_endpoints(com, axis, vessel_contour)
        if min_pt_vessel is None or max_pt_vessel is None:
            continue
        external_diameter = np.linalg.norm(max_pt_vessel - min_pt_vessel)

        sclerotic_idx = 1 - (internal_diameter / external_diameter)
        sclerotic_indices.append(sclerotic_idx)

    sclerotic_indices = np.array(sclerotic_indices)

    results = {
        'median': np.median(sclerotic_indices),
        'mean': np.mean(sclerotic_indices),
        'std_dev': np.std(sclerotic_indices),
        'min': np.min(sclerotic_indices),
        'max': np.max(sclerotic_indices)
    }

    return results

def find_contour_intersection(point, vector, contour_pts, buffer=1e-6):
    axis_unit = vector / np.linalg.norm(vector)
    intersections = []

    for i in range(len(contour_pts)):
        p1 = contour_pts[i]
        p2 = contour_pts[(i + 1) % len(contour_pts)]
        intersection = line_intersection(point - 10000 * axis_unit, point + 10000 * axis_unit, p1, p2)

        min_bound = np.minimum(p1, p2) - buffer
        max_bound = np.maximum(p1, p2) + buffer

        if np.all(np.isfinite(intersection)) and np.all(min_bound <= intersection) and np.all(intersection <= max_bound):
            intersections.append(intersection)

    if not intersections:
        return None

    distances = [np.linalg.norm(np.array(intersection) - point) for intersection in intersections]
    nearest_pt = intersections[np.argmin(distances)]
    
    return np.array(nearest_pt)

def smooth_contour(contour, smooth_factor=20, num_points=1000):
    tck, u = splprep([contour[:, 0], contour[:, 1]], s=smooth_factor, per=True)
    u_fine = np.linspace(0, 1, num_points)
    x_smooth, y_smooth = splev(u_fine, tck)
    smooth_contour = np.vstack((x_smooth, y_smooth)).T
    return smooth_contour

def tangent(line, i):
    pts = line[max(0, i-2):min(len(line), i+3)]
    t = np.polyfit(pts[:,0], pts[:,1], 1)
    tangent_line = np.array([1, t[0]])  # slope form
    tangent_line /= np.linalg.norm(tangent_line)  # make it a unit vector
    return tangent_line

def calculate_thickness_tangent(lumen_segmentation, vessel_segmentation, smooth_factor=20, num_lines=100):
    lumen_contours, _ = cv2.findContours(lumen_segmentation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    vessel_contours, _ = cv2.findContours(vessel_segmentation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    lumen_contour = lumen_contours[0].reshape(-1, 2)
    vessel_contour = vessel_contours[0].reshape(-1, 2)
    
    smooth_lumen_contour = smooth_contour(lumen_contour, smooth_factor)
    
    if not np.array_equal(smooth_lumen_contour[0], smooth_lumen_contour[-1]):
        smooth_lumen_contour = np.vstack([smooth_lumen_contour, smooth_lumen_contour[0]])
    
    M = cv2.moments(lumen_contour)
    if M["m00"] != 0:
        lumen_center = np.array([M["m10"] / M["m00"], M["m01"] / M["m00"]])
    else:
        lumen_center = np.mean(lumen_contour, axis=0)
    
    thicknesses = []
    lines = []
    
    n_points = len(smooth_lumen_contour)
    step_size = n_points // num_lines

    for i in range(0, n_points, step_size):
        point = smooth_lumen_contour[i]
        
        tangent_line = tangent(smooth_lumen_contour, i)
        
        thickness_line_vector = np.array([-tangent_line[1], tangent_line[0]])
        
        direction_vector = point - lumen_center
        if np.dot(thickness_line_vector, direction_vector) < 0:
            thickness_line_vector = -thickness_line_vector
        
        lumen_intersection = find_contour_intersection(point, thickness_line_vector, lumen_contour)
        vessel_intersection = find_contour_intersection(point, thickness_line_vector, vessel_contour)
        
        if lumen_intersection is None or vessel_intersection is None:
            continue
        
        thickness = np.linalg.norm(vessel_intersection - lumen_intersection)
        thicknesses.append(thickness)
        
        lines.append((lumen_intersection, vessel_intersection))
    
    metrics = {
        "median": np.median(thicknesses),
        "mean": np.mean(thicknesses),
        "std": np.std(thicknesses),
        "min": np.min(thicknesses),
        "max": np.max(thicknesses)
    }

    return metrics

def calculate_thickness_radii(vessel_segmentation, lumen_segmentation):
    com = get_centroid(lumen_segmentation)
    lumen_contour = find_contours(lumen_segmentation)
    vessel_contour = find_contours(vessel_segmentation)

    thicknesses = []

    for angle in range(360):
        radians = np.deg2rad(angle)
        axis = np.array([np.cos(radians), np.sin(radians)])

        min_pt_lumen, max_pt_lumen = find_constrained_endpoints(com, axis, lumen_contour)
        if min_pt_lumen is None or max_pt_lumen is None:
            continue
        lumen_radius = np.linalg.norm(max_pt_lumen - min_pt_lumen)

        min_pt_vessel, max_pt_vessel = find_constrained_endpoints(com, axis, vessel_contour)
        if min_pt_vessel is None or max_pt_vessel is None:
            continue
        vessel_radius = np.linalg.norm(max_pt_vessel - min_pt_vessel)

        thickness = vessel_radius - lumen_radius
        thicknesses.append(thickness)

    metrics = {
        "median": np.median(thicknesses),
        "mean": np.mean(thicknesses),
        "std": np.std(thicknesses),
        "min": np.min(thicknesses),
        "max": np.max(thicknesses)
    }

    return metrics

# Define the conversion factor
PIXEL_TO_MICRON_SQUARED = 0.0484

def calculate_areas_and_ratio(vessel_segmentation, lumen_segmentation):
    vessel_area_pixels = np.sum(vessel_segmentation)
    lumen_area_pixels = np.sum(lumen_segmentation)
    wall_area_pixels = vessel_area_pixels - lumen_area_pixels
    
    # Convert areas to square microns
    vessel_area_microns = vessel_area_pixels * PIXEL_TO_MICRON_SQUARED
    lumen_area_microns = lumen_area_pixels * PIXEL_TO_MICRON_SQUARED
    wall_area_microns = wall_area_pixels * PIXEL_TO_MICRON_SQUARED
    
    area_ratio = wall_area_microns / lumen_area_microns if lumen_area_microns > 0 else float('inf')
    
    return lumen_area_microns, wall_area_microns, area_ratio

In [3]:
# Load your model
name = 'Attention_Unfrozen_EfficientNetV2L_v0.7_try2'
root = '/home/jjlou/Jerry/wsi-arterio/arteriosclerotic_vessel_detection_and_fine_segmentation/Vessel_WallsLumen_Segmentation/data_test_v2'

custom_objects = {
    'dice_all': jerry_metrics.dice_metric(cls=1),
    'hausdorff_all': jerry_metrics.hausdorff_metric(cls=1),
    'focal_dice_like_loss_multiclass_weighted': jerry_losses.focal_dice_like_loss_multiclass_weighted
}

model_path = f'{root}/models_raw/{name}.hdf5'
model = models.load_model(model_path, custom_objects=custom_objects)

# Load your test dataset
path = f'{root}/VasM_test'
test = tf.data.Dataset.load(path)

In [4]:
# Initialize lists to store results
sclerotic_index_medians = []
sclerotic_index_means = []
sclerotic_index_stds = []
sclerotic_index_minimums = []
sclerotic_index_maximums = []

thickness_tangent_medians = []
thickness_tangent_means = []
thickness_tangent_stds = []
thickness_tangent_minimums = []
thickness_tangent_maximums = []

thickness_radii_medians = []
thickness_radii_means = []
thickness_radii_stds = []
thickness_radii_minimums = []
thickness_radii_maximums = []

lumen_areas = []  
wall_areas = []   
area_ratios = []

# Process each segmentation in the dataset
count = 0
for i, p in test:
    p = p.numpy()
    vessel = p > 0
    lumen = p == 2
    vessel_segmentation = np.squeeze(vessel).astype(np.uint8)
    lumen_segmentation = np.squeeze(lumen).astype(np.uint8)
    
    try:
        # Calculate sclerotic index metrics
        sclerotic_metrics = sclerotic_index(vessel_segmentation, lumen_segmentation)
        sclerotic_index_medians.append(sclerotic_metrics["median"])
        sclerotic_index_means.append(sclerotic_metrics["mean"])
        sclerotic_index_stds.append(sclerotic_metrics["std_dev"])
        sclerotic_index_minimums.append(sclerotic_metrics["min"])
        sclerotic_index_maximums.append(sclerotic_metrics["max"])

        # Calculate vessel wall thickness using tangent method
        thickness_tangent_metrics = calculate_thickness_tangent(lumen_segmentation, vessel_segmentation)
        thickness_tangent_medians.append(thickness_tangent_metrics["median"])
        thickness_tangent_means.append(thickness_tangent_metrics["mean"])
        thickness_tangent_stds.append(thickness_tangent_metrics["std"])
        thickness_tangent_minimums.append(thickness_tangent_metrics["min"])
        thickness_tangent_maximums.append(thickness_tangent_metrics["max"])

        # Calculate vessel wall thickness using radii method
        thickness_radii_metrics = calculate_thickness_radii(vessel_segmentation, lumen_segmentation)
        thickness_radii_medians.append(thickness_radii_metrics["median"])
        thickness_radii_means.append(thickness_radii_metrics["mean"])
        thickness_radii_stds.append(thickness_radii_metrics["std"])
        thickness_radii_minimums.append(thickness_radii_metrics["min"])
        thickness_radii_maximums.append(thickness_radii_metrics["max"])

        # Calculate areas (in square microns) and area ratio
        lumen_area, wall_area, area_ratio = calculate_areas_and_ratio(vessel_segmentation, lumen_segmentation)
        lumen_areas.append(lumen_area)
        wall_areas.append(wall_area)
        area_ratios.append(area_ratio)

    except:
        print(f'This output number didnt process correctly:{count}')
    
    count += 1

# Create a dictionary from the results
metrics_dict = {
    'sclerotic_index_median': sclerotic_index_medians,
    'sclerotic_index_mean': sclerotic_index_means,
    'sclerotic_index_std_dev': sclerotic_index_stds,
    'sclerotic_index_min': sclerotic_index_minimums,
    'sclerotic_index_max': sclerotic_index_maximums,
    'thickness_tangent_median': thickness_tangent_medians,
    'thickness_tangent_mean': thickness_tangent_means,
    'thickness_tangent_std': thickness_tangent_stds,
    'thickness_tangent_min': thickness_tangent_minimums,
    'thickness_tangent_max': thickness_tangent_maximums,
    'thickness_radii_median': thickness_radii_medians,
    'thickness_radii_mean': thickness_radii_means,
    'thickness_radii_std': thickness_radii_stds,
    'thickness_radii_min': thickness_radii_minimums,
    'thickness_radii_max': thickness_radii_maximums,
    'lumen_area_microns': lumen_areas,     
    'wall_area_microns': wall_areas,       
    'area_ratio': area_ratios
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(metrics_dict)
df.to_csv(f'{root}/models_results/{name}_Sclerotic.Index_Vessel.Wall.Thickness_Areas.Ratio.csv', index=False)

print("Metrics calculation and saving completed.")

Metrics calculation and saving completed.
